In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
df = pd.read_csv('letter-recognition.data',header=None)

In [3]:
train_df = df[:16000]
test_df = df[16000:]

In [4]:
X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [6]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        
        outputs = []
        for xi in x_inputs:
            output = net.activate(xi)
            outputs.append(output)
        
        
        px_outputs = softmax(np.array(outputs).reshape(samplesize, number_of_classification), axis=1)
        # the index of maximum in each line
        pred_outputs = np.argmax(px_outputs, axis = 1)
        # replace index to real number, from higher number to lower number!!
        for i in range(number_of_classification):
            index = number_of_classification - i - 1
            pred_outputs[pred_outputs==index] = numbers[index]
        
        real_outputs = np.array(x_outputs).reshape(samplesize,)
        
        acc = np.sum(pred_outputs == real_outputs)/samplesize

        genome.fitness = acc

In [7]:
numbers = [*range(26)] # number 2 > number 1
number_of_classification = len(numbers)


list_y = y.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(list_y)) if list_y[i] == digit]
    digits_indexes.extend(li)

samplesize = X[digits_indexes].shape[0]
x_inputs =[tuple(c) for c in X[digits_indexes].tolist()]
x_outputs = [tuple(c) for c in y[digits_indexes].reshape(samplesize,1).tolist()]

In [8]:
def run(config_file):
    # Load configuration.
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file,
    )

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # add a stdout reporter to show progress in the terminal
    reporter = neat.StdOutReporter(False)
    p.add_reporter(reporter)
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #checkpointer = neat.Checkpointer(100)
    #p.add_reporter(checkpointer)
    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 30)

    return [stats, winner]


samplesize = len(x_outputs)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter26")
[stats, winner] = run(config_path)

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)


 ****** Running generation 0 ****** 

Population's average fitness: 0.03789 stdev: 0.01042
Best fitness: 0.08225 - size: (26, 42) - species 11 - id 116
Average adjusted fitness: 0.031
Mean genetic distance 2.710, standard deviation 0.253
Population of 199 members in 38 species
Total extinctions: 0
Generation time: 157.481 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.04089 stdev: 0.01025
Best fitness: 0.08225 - size: (26, 42) - species 8 - id 116
Average adjusted fitness: 0.033
Mean genetic distance 2.793, standard deviation 0.291
Population of 202 members in 40 species
Total extinctions: 0
Generation time: 154.346 sec (155.913 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.04220 stdev: 0.01102
Best fitness: 0.10587 - size: (27, 42) - species 36 - id 374
Average adjusted fitness: 0.037
Mean genetic distance 2.810, standard deviation 0.307
Population of 202 members in 40 species
Total extinctions: 0
Generation time: 156.486 

Population's average fitness: 0.05438 stdev: 0.01683
Best fitness: 0.10587 - size: (27, 42) - species 36 - id 374
Average adjusted fitness: 0.031
Mean genetic distance 3.054, standard deviation 0.431
Population of 202 members in 23 species
Total extinctions: 0
Generation time: 151.033 sec (160.910 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.05325 stdev: 0.01774
Best fitness: 0.10587 - size: (27, 42) - species 36 - id 374
Average adjusted fitness: 0.036
Mean genetic distance 3.057, standard deviation 0.435
Population of 201 members in 23 species
Total extinctions: 0
Generation time: 154.446 sec (159.213 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.05326 stdev: 0.01721
Best fitness: 0.10587 - size: (27, 42) - species 36 - id 374
Average adjusted fitness: 0.040
Mean genetic distance 3.053, standard deviation 0.437
Population of 199 members in 23 species
Total extinctions: 0
Generation time: 166.728 sec (159.491 avera